# PPO in Stable Baselines

In single-agent PPO, `MlpPolicy` was used in `PPO1` as follows:

```
model = PPO1(MlpPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

```

`MlpPolicy` is found in `stable_baselines/common/policies.py`, inheriting `FeedForwardPolicy`, which inherits from `ActorCriticPolicy`.

In `FeedForwardPolicy`'s `__init__`, there contains the following:
```
if net_arch is None:
    if layers is None:
        layers = [64, 64]
    net_arch = [dict(vf=layers, pi=layers)]

with tf.variable_scope("model", reuse=reuse):
    if feature_extraction == "cnn":
        pi_latent = vf_latent = cnn_extractor(self.processed_obs, **kwargs)
    else:
        pi_latent, vf_latent = mlp_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)

    self._value_fn = linear(vf_latent, 'vf', 1)

    self._proba_distribution, self._policy, self.q_value = \
        self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)
```

Since `MlpPolicy` uses `feature_extraction="mlp"`, look into `mlp_extractor` (here)[https://github.com/hill-a/stable-baselines/blob/master/stable_baselines/common/policies.py].

`mlp_extractor` constructs a MLP that receive observations as input and outputs a latent representation for the policy and a value network. Amount and size of hidden layers and how many shared between policy and value network can be spcified using `net_arch`.

In `mlp_extractor`, it iterates through `net_arch` and creates layers, specifically using `latent = act_fun(linear(latent, ...))`. Therfore, look into `act_fun` and `linear`, which belongs in stable_baselines.common.tf_layers.

`FeedForwardPolicy`'s default for `act_fun` is `tf.tanh`. Linear contains:

```
def linear(input_tensor, scope, n_hidden, *, init_scale=1.0, init_bias=0.0):
    """
    Creates a fully connected layer for TensorFlow
    :param input_tensor: (TensorFlow Tensor) The input tensor for the fully connected layer
    :param scope: (str) The TensorFlow variable scope
    :param n_hidden: (int) The number of hidden neurons
    :param init_scale: (int) The initialization scale
    :param init_bias: (int) The initialization offset bias
    :return: (TensorFlow Tensor) fully connected layer
    """
    with tf.variable_scope(scope):
        n_input = input_tensor.get_shape()[1].value
        weight = tf.get_variable("w", [n_input, n_hidden], initializer=ortho_init(init_scale))
        bias = tf.get_variable("b", [n_hidden], initializer=tf.constant_initializer(init_bias))
        return tf.matmul(input_tensor, weight) + bias
```

Therefore, to transform this model into a Bayesian neural network, the linear layer needs to be changed into DenseVariational instead of a linear layer. We can do this by modifying `FeedForwardPolicy` (which `MlpPolicy` inherits) with a new `bnn_extractor`, then creating a `BnnPolicy` to replace `MlpPolicy`.

In [157]:
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers
from tensorflow.keras.layers import Layer

import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfp.__version__

'0.8.0'

In [158]:
# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])

# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t, scale=1),
            reinterpreted_batch_ndims=1)),
    ])

In [159]:
def bnn_extractor(flat_observations, net_arch, act_fun):
    """
    Constructs an variational layer that receives observations as an input and outputs a latent representation for the policy and
    a value network. The ``net_arch`` parameter allows to specify the amount and size of the hidden layers and how many
    of them are shared between the policy network and the value network. It is assumed to be a list with the following
    structure:
    1. An arbitrary length (zero allowed) number of integers each specifying the number of units in a shared layer.
       If the number of ints is zero, there will be no shared layers.
    2. An optional dict, to specify the following non-shared layers for the value network and the policy network.
       It is formatted like ``dict(vf=[<value layer sizes>], pi=[<policy layer sizes>])``.
       If it is missing any of the keys (pi or vf), no non-shared layers (empty list) is assumed.
    For example to construct a network with one shared layer of size 55 followed by two non-shared layers for the value
    network of size 255 and a single non-shared layer of size 128 for the policy network, the following layers_spec
    would be used: ``[55, dict(vf=[255, 255], pi=[128])]``. A simple shared network topology with two layers of size 128
    would be specified as [128, 128].
    :param flat_observations: (tf.Tensor) The observations to base policy and value function on.
    :param net_arch: ([int or dict]) The specification of the policy and value networks.
        See above for details on its formatting.
    :param act_fun: (tf function) The activation function to use for the networks.
    :return: (tf.Tensor, tf.Tensor) latent_policy, latent_value of the specified network.
        If all layers are shared, then ``latent_policy == latent_value``
    """
    latent = flat_observations
    policy_only_layers = []  # Layer sizes of the network that only belongs to the policy network
    value_only_layers = []  # Layer sizes of the network that only belongs to the value network

    # Iterate through the shared layers and build the shared parts of the network
    for idx, layer in enumerate(net_arch):
        if isinstance(layer, int):  # Check that this is a shared layer
            layer_size = layer
#             latent = act_fun(linear(latent, "shared_fc{}".format(idx), layer_size, init_scale=np.sqrt(2)))
            latent = act_fun(tfp.layers.DenseVariational(layer_size, posterior_mean_field, prior_trainable, kl_weight=1))
        else:
            assert isinstance(layer, dict), "Error: the net_arch list can only contain ints and dicts"
            if 'pi' in layer:
                assert isinstance(layer['pi'], list), "Error: net_arch[-1]['pi'] must contain a list of integers."
                policy_only_layers = layer['pi']

            if 'vf' in layer:
                assert isinstance(layer['vf'], list), "Error: net_arch[-1]['vf'] must contain a list of integers."
                value_only_layers = layer['vf']
            break  # From here on the network splits up in policy and value network

    # Build the non-shared part of the network
    latent_policy = latent
    latent_value = latent
    for idx, (pi_layer_size, vf_layer_size) in enumerate(zip_longest(policy_only_layers, value_only_layers)):
        if pi_layer_size is not None:
            assert isinstance(pi_layer_size, int), "Error: net_arch[-1]['pi'] must only contain integers."
#             latent_policy = act_fun(linear(latent_policy, "pi_fc{}".format(idx), pi_layer_size, init_scale=np.sqrt(2)))
            latent_policy = act_fun(tfp.layers.DenseVariational(pi_layer_size, posterior_mean_field, prior_trainable, kl_weight=1)(latent_policy))

        if vf_layer_size is not None:
            assert isinstance(vf_layer_size, int), "Error: net_arch[-1]['vf'] must only contain integers."
#             latent_value = act_fun(linear(latent_value, "vf_fc{}".format(idx), vf_layer_size, init_scale=np.sqrt(2)))
            latent_value = act_fun(tfp.layers.DenseVariational(vf_layer_size, posterior_mean_field, prior_trainable, kl_weight=1)(latent_value))

    return latent_policy, latent_value

In [160]:
class FeedForwardPolicy(ActorCriticPolicy):
    """
    Policy object that implements actor critic, using a feed forward neural network.
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param layers: ([int]) (deprecated, use net_arch instead) The size of the Neural network for the policy
        (if None, default to [64, 64])
    :param net_arch: (list) Specification of the actor-critic policy network architecture (see mlp_extractor
        documentation for details).
    :param act_fun: (tf.func) the activation function to use in the neural network.
    :param cnn_extractor: (function (TensorFlow Tensor, ``**kwargs``): (TensorFlow Tensor)) the CNN feature extraction
    :param feature_extraction: (str) The feature extraction type ("cnn" or "mlp")
    :param kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, layers=None, net_arch=None,
                 act_fun=tf.tanh, cnn_extractor=nature_cnn, feature_extraction="cnn", **kwargs):
        super(FeedForwardPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse,
                                                scale=(feature_extraction == "cnn"))

        self._kwargs_check(feature_extraction, kwargs)

        if layers is not None:
            warnings.warn("Usage of the `layers` parameter is deprecated! Use net_arch instead "
                          "(it has a different semantics though).", DeprecationWarning)
            if net_arch is not None:
                warnings.warn("The new `net_arch` parameter overrides the deprecated `layers` parameter!",
                              DeprecationWarning)

        if net_arch is None:
            if layers is None:
                layers = [64, 64]
            net_arch = [dict(vf=layers, pi=layers)]

        with tf.variable_scope("model", reuse=reuse):
            if feature_extraction == "cnn":
                pi_latent = vf_latent = cnn_extractor(self.processed_obs, **kwargs)
            elif feature_extraction == "bnn":
                pi_latent, vf_latent = bnn_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)
            else:
                pi_latent, vf_latent = mlp_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)

            self._value_fn = linear(vf_latent, 'vf', 1)

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [161]:
import warnings
from itertools import zip_longest
from abc import ABC, abstractmethod

import numpy as np
import tensorflow as tf
from gym.spaces import Discrete

from stable_baselines.common.tf_util import batch_to_seq, seq_to_batch
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc, lstm
from stable_baselines.common.distributions import make_proba_dist_type, CategoricalProbabilityDistribution, \
    MultiCategoricalProbabilityDistribution, DiagGaussianProbabilityDistribution, BernoulliProbabilityDistribution
from stable_baselines.common.input import observation_input
from stable_baselines.common.policies import nature_cnn

In [162]:
class BnnPolicy(FeedForwardPolicy):
    """
    Policy object that implements actor critic, using a Bayesian neural net (2 layers of 64)
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param _kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **_kwargs):
        super(BnnPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse,
                                        feature_extraction="bnn", **_kwargs)

# Single-Agent PPO with BNN

In [163]:
#!/usr/bin/env python3

# Train single CPU PPO1 on slimevolley.
# Should solve it (beat existing AI on average over 1000 trials) in 3 hours on single CPU, within 3M steps.

import os
import gym
import slimevolleygym
from slimevolleygym import SurvivalRewardEnv

from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback

NUM_TIMESTEPS = int(5e6)
SEED = 721
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000
LOGDIR = "bnn_ppo1" # moved to zoo afterwards.

logger.configure(folder=LOGDIR)

env = gym.make("SlimeVolley-v0")
env.seed(SEED)

Logging to bnn_ppo1


[721]

In [164]:
# take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
model = PPO1(BnnPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

model.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

model.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()

********** Iteration 0 ************


C:\Users\notes\Anaconda3\envs\slime-rl\lib\site-packages\stable_baselines\common\callbacks.py:287: UserWarning: Training and eval env are not of the same type<SlimeVolleyEnv<SlimeVolley-v0>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x000001C9426233C8>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00074 |       0.00000 |       1.01718 |       0.00014 |       2.07934
     3.04e-05 |       0.00000 |       0.88093 |       0.00028 |       2.07920
     -0.00026 |       0.00000 |       0.87572 |       0.00045 |       2.07903
      0.00029 |       0.00000 |       0.76330 |       0.00054 |       2.07893
     -0.00033 |       0.00000 |       0.73383 |       0.00065 |       2.07883
     -0.00106 |       0.00000 |       0.60832 |       0.00084 |       2.07862
     -0.00165 |       0.00000 |       0.54448 |       0.00106 |       2.07842
      0.00029 |       0.00000 |       0.49235 |       0.00124 |       2.07823
     -0.00087 |       0.00000 |       0.48766 |       0.00134 |       2.07813
      0.00092 |       0.00000 |       0.43227 |       0.00147 |       2.07802
Evaluating losses...
     -0.00190 |       0.00000 |       0.41145 |       0.00165 |       2.07782
-----------------------------

      0.00083 |       0.00000 |       0.05020 |       0.00456 |       2.07654
     -0.00032 |       0.00000 |       0.05025 |       0.00439 |       2.07652
     -0.00081 |       0.00000 |       0.04994 |       0.00462 |       2.07635
Evaluating losses...
      0.00092 |       0.00000 |       0.05043 |       0.00448 |       2.07632
----------------------------------
| EpLenMean       | 564          |
| EpRewMean       | -4.93        |
| EpThisIter      | 7            |
| EpisodesSoFar   | 43           |
| TimeElapsed     | 38.9         |
| TimestepsSoFar  | 24576        |
| ev_tdlam_before | -0.0109      |
| loss_ent        | 2.0763159    |
| loss_kl         | 0.0044785896 |
| loss_pol_entpen | 0.0          |
| loss_pol_surr   | 0.0009182282 |
| loss_vf_loss    | 0.05043229   |
----------------------------------
********** Iteration 6 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00154 |       0.00000 |       0.05140 |  

********** Iteration 11 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -2.73e-05 |       0.00000 |       0.05114 |       0.00394 |       2.07346
     1.51e-05 |       0.00000 |       0.05079 |       0.00401 |       2.07348
      0.00199 |       0.00000 |       0.05112 |       0.00391 |       2.07330
     -0.00140 |       0.00000 |       0.05097 |       0.00395 |       2.07327
     -0.00157 |       0.00000 |       0.05087 |       0.00435 |       2.07307
     -0.00209 |       0.00000 |       0.05091 |       0.00419 |       2.07221
     -0.00035 |       0.00000 |       0.05038 |       0.00442 |       2.07200
    -3.37e-05 |       0.00000 |       0.05076 |       0.00467 |       2.07215
      0.00111 |       0.00000 |       0.05128 |       0.00460 |       2.07206
      0.00123 |       0.00000 |       0.05153 |       0.00443 |       2.07171
Evaluating losses...
      0.00044 |       0.00000 |       0.05117 |       0.00454 |       

...

********** Iteration 726 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     1.60e-05 |       0.00000 |       0.01931 |       0.00078 |       1.60519
      0.00028 |       0.00000 |       0.01925 |       0.00084 |       1.60412
      0.00076 |       0.00000 |       0.01912 |       0.00083 |       1.60242
     -0.00064 |       0.00000 |       0.01919 |       0.00084 |       1.60172
     3.19e-05 |       0.00000 |       0.01913 |       0.00092 |       1.60000
     -0.00018 |       0.00000 |       0.01919 |       0.00082 |       1.59901
     -0.00015 |       0.00000 |       0.01915 |       0.00089 |       1.59936
     -0.00064 |       0.00000 |       0.01901 |       0.00092 |       1.59711
      0.00044 |       0.00000 |       0.01911 |       0.00084 |       1.59710
      0.00023 |       0.00000 |       0.01910 |       0.00093 |       1.59603
Evaluating losses...
      0.00045 |       0.00000 |       0.01906 |       0.00092 |      

      0.00117 |       0.00000 |       0.01366 |       0.00093 |       1.60199
     -0.00077 |       0.00000 |       0.01356 |       0.00102 |       1.60160
      0.00037 |       0.00000 |       0.01358 |       0.00090 |       1.60074
Evaluating losses...
     8.87e-05 |       0.00000 |       0.01353 |       0.00089 |       1.60091
----------------------------------
| EpLenMean       | 646          |
| EpRewMean       | -4.79        |
| EpThisIter      | 7            |
| EpisodesSoFar   | 4753         |
| TimeElapsed     | 5.47e+03     |
| TimestepsSoFar  | 2998272      |
| ev_tdlam_before | 0.866        |
| loss_ent        | 1.6009091    |
| loss_kl         | 0.0008851342 |
| loss_pol_entpen | 0.0          |
| loss_pol_surr   | 8.866016e-05 |
| loss_vf_loss    | 0.01353225   |
----------------------------------
********** Iteration 732 ************
Eval num_timesteps=2998272, episode_reward=-4.40 +/- 0.92
Episode length: 619.40 +/- 109.86
New best mean reward!
Optimizing...
     pol_su

********** Iteration 737 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00091 |       0.00000 |       0.01591 |       0.00068 |       1.59015
      0.00034 |       0.00000 |       0.01561 |       0.00070 |       1.59053
      0.00016 |       0.00000 |       0.01550 |       0.00068 |       1.59093
     -0.00019 |       0.00000 |       0.01530 |       0.00068 |       1.59224
     -0.00051 |       0.00000 |       0.01531 |       0.00069 |       1.59064
      0.00038 |       0.00000 |       0.01497 |       0.00074 |       1.59190
      0.00123 |       0.00000 |       0.01485 |       0.00072 |       1.59154
      0.00092 |       0.00000 |       0.01481 |       0.00072 |       1.59217
      0.00034 |       0.00000 |       0.01518 |       0.00071 |       1.59097
     -0.00043 |       0.00000 |       0.01474 |       0.00071 |       1.59236
Evaluating losses...
      0.00035 |       0.00000 |       0.01495 |       0.00071 |      

      0.00030 |       0.00000 |       0.01436 |       0.00074 |       1.60488
     -0.00049 |       0.00000 |       0.01435 |       0.00074 |       1.60507
     -0.00118 |       0.00000 |       0.01450 |       0.00083 |       1.60581
Evaluating losses...
      0.00045 |       0.00000 |       0.01407 |       0.00081 |       1.60715
-----------------------------------
| EpLenMean       | 627           |
| EpRewMean       | -4.87         |
| EpThisIter      | 6             |
| EpisodesSoFar   | 4825          |
| TimeElapsed     | 5.78e+03      |
| TimestepsSoFar  | 3043328       |
| ev_tdlam_before | 0.865         |
| loss_ent        | 1.6071502     |
| loss_kl         | 0.00081473275 |
| loss_pol_entpen | 0.0           |
| loss_pol_surr   | 0.00044506788 |
| loss_vf_loss    | 0.014065403   |
-----------------------------------
********** Iteration 743 ************


KeyboardInterrupt: 

In [165]:
obs = env.reset()
done = False
total_reward = 0

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()

print("score:", total_reward)

score: -5
